In [ ]:
pip install azure-cognitiveservices-vision-customvision

In [ ]:
pip install gradio_modal

In [ ]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from gradio_modal import Modal
import cv2
from io import BytesIO
import requests
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from loguru import logger
import numpy as np
import io
import time

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Initialize the prediction client
ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
 
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}

reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}

def split_hangul(word):
    # 한글 초성, 중성, 종성 리스트
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
 
    separated = []
    for char in word:
        if '가' <= char <= '힣':  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성
            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])       # 중성 추가
            if final_consonants[final]:                   # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가
 
    return separated

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 음성 멈춤 기능을 위한 전역 변수
is_speaking = False

# 음성을 멈추는 함수
def stop_speech():
    global is_speaking
    if is_speaking:
        speech_synthesizer.stop_speaking_async()
        is_speaking = False
    return "Speech stopped."

# 채팅
def chat_with_openai(user_input, chat_history):
    # 수어가 있는 경우
    try:
        global is_speaking
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정

        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # 출처와 관련된 내용 확인
        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')  # 이미지 URL이 여러 개일 수 있음

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        is_speaking = True
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, '', video_url, image_urls, gr.update(visible=True), gr.update(visible=False)

    # 지문자로 대체하는 경우
    except Exception as e:
        # 오류 메시지 대체
        fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        chat_history.append((user_input, fallback_message))

        return chat_history, '', None, [], gr.update(visible=False), gr.update(visible=True)


def draw_boxes(image, predictions):
    """Draw bounding boxes on the image based on predictions"""
    img = image.copy()
    for pred in predictions:
        if pred.probability > 0.9 :
            color = (255, 0, 0)
            box = pred.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])
        
        # Draw rectangle
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
        
        # Add label with confidence score
            label = f"{pred.tag_name}: {pred.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img, pred.tag_name

def click_sign_send(sign_word):
    split_sign = split_hangul(sign_word)
    logger.info(sign_word + " " + str(split_sign))

    gallery_images = []
    sign_confirmed = []

    for sign in split_sign:
        sign_file_name = reverse_hangul_dict[sign]
        gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
        sign_confirmed.append(False)
    
    return gr.update(visible=False), gr.update(visible=True), gallery_images, sign_confirmed

# Custom Vision
def process_frame(frame, gallery_origin_images, sign_word, sign_confirmed):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame, tag_name = draw_boxes(frame, results.predictions)

        gallery_images = []

        split_sign = split_hangul(sign_word)
        # logger.info(sign_confirmed)
        sign_confirmed = sign_confirmed.replace("[","").replace("]", "")
        # logger.info(sign_confirmed)

        sign_confirmed_list = sign_confirmed.split(',')
        sign_confirmed_str_list = []

        for sign_confirmed in sign_confirmed_list:
            sign_confirmed = sign_confirmed.replace("'","").replace('"', "").strip()
            sign_confirmed_str_list.append(sign_confirmed)
            # logger.info(sign_confirmed)

        logger.info(sign_confirmed_str_list)

        # for confirmed in sign_confirmed_str_list:
        #     if confirmed == "True":
        #         logger.info(str(sign_confirmed_str_list))

        confirmed_result= []
        for i in range(len(split_sign)):
            confirmed_result.append("False")


        i = 0
        for sign in split_sign:
            sign_file_name = reverse_hangul_dict[sign]
            confirmed = (sign_confirmed_str_list[i] == 'True')
            logger.info(confirmed)            
            if confirmed:
                gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                confirmed_result[i] = "True"
            else:
                if sign_file_name == tag_name:
                    gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                    confirmed_result[i] = "True"
                else :    
                    gallery_images.append(f"./images/Basic/{sign_file_name}.jpg") 
                    confirmed_result[i] = "False"
            i = i+1

        return annotated_frame, gallery_images, str(confirmed_result)
    
    except Exception as e:
        logger.error(f"Error during prediction: {e}")
        return frame, gallery_origin_images, sign_confirmed


# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            stop_button = gr.Button("Stop Speech")
        with gr.Column(visible=False) as column1:   # 기본적으로 숨김
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)
        with gr.Column(visible=False) as column2:   # 기본적으로 숨김
            with gr.Row():
                sign_input_textbox = gr.Textbox(label="지문자 입력 (2글자 단어)", value="", scale=7)
                sign_confirmed_textbox = gr.Textbox(show_label=False, visible=True)
                toggle_button = gr.Button("지문자 확인", scale=1)
            with gr.Row():  # Row layout for input and output
                gallery = gr.Gallery(columns=[6], rows=[1], show_label=False, show_share_button=False, show_download_button=False, interactive=False, show_fullscreen_button=False, height=130)
            with gr.Row():
                webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                webcam_output = gr.Image(label="Detected Objects")
            show_btn = gr.Button("힌트")
            webcam_input.stream(process_frame, inputs=[webcam_input, gallery, sign_input_textbox, sign_confirmed_textbox], outputs=[webcam_output, gallery, sign_confirmed_textbox])
            demo.title = "Azure Custom Vision Object Detection"
            demo.description = "Real-time object detection using Azure Custom Vision"
        
    with Modal(visible=False) as modal:  # 지문자 힌트 보여주기
            gr.Textbox("이미지가 보여질 공간")


    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, user_input, video_display, image_display, column1, column2])
    clear_button.click(
        lambda: ([], []),  # chatbot과 state를 모두 초기화
        inputs=None,
        outputs=[chatbot, state]  # chatbot과 state 모두 업데이트
    )
    stop_button.click(stop_speech)  # 음성을 멈추기
    show_btn.click(lambda: Modal(visible=True), None, modal)  # 지문자 힌트 보여주기
    toggle_button.click(fn=click_sign_send, inputs=[sign_input_textbox], outputs=[column1, column2, gallery, sign_confirmed_textbox])

# 실행
demo.launch(share=False)

In [ ]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from gradio_modal import Modal
import cv2
from io import BytesIO
import requests
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from loguru import logger
import numpy as np
import io
import time

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Initialize the prediction client
ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
 
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}

reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}

def split_hangul(word):
    # 한글 초성, 중성, 종성 리스트
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
 
    separated = []
    for char in word:
        if '가' <= char <= '힣':  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성
            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])       # 중성 추가
            if final_consonants[final]:                   # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가
 
    return separated

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 음성 멈춤 기능을 위한 전역 변수
is_speaking = False

# 음성을 멈추는 함수
def stop_speech():
    global is_speaking
    if is_speaking:
        speech_synthesizer.stop_speaking_async()
        is_speaking = False
    return "Speech stopped."

# 채팅
def chat_with_openai(user_input, chat_history):
    # 수어가 있는 경우
    try:
        global is_speaking
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정

        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # 출처와 관련된 내용 확인
        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')  # 이미지 URL이 여러 개일 수 있음

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        is_speaking = True
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, '', video_url, image_urls, gr.update(visible=True), gr.update(visible=False)

    # 지문자로 대체하는 경우
    except Exception as e:
        # 오류 메시지 대체
        fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        chat_history.append((user_input, fallback_message))

        return chat_history, '', None, [], gr.update(visible=False), gr.update(visible=True)


def draw_boxes(image, predictions):
    """가장 확률이 높은 객체만 경계 상자를 그리도록 수정"""
    img = image.copy()
    
    # 예측 결과 중 확률이 가장 높은 하나를 선택
    if predictions:
        highest_prediction = max(predictions, key=lambda p: p.probability)
        
        # if first_char_is_inprogress:
        # logger.info(str(highest_prediction.tag_name) + " " + str(highest_prediction.probability))
            # if highest_prediction.tag_name == first_char and highest_prediction.probability > 0.7:
                # global first_char_succeed
                # first_char_succeed = True
                # print("First character detected successfully!")
        
        # 확률이 0.5 이상인 객체만 선택
        if highest_prediction.probability > 0.5:
            color = (255, 0, 0)  # 경계 상자 색상 (빨간색)
            box = highest_prediction.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])

            # 경계 상자 그리기
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
            
            # 라벨과 확률 텍스트 추가
            label = f"{highest_prediction.tag_name}: {highest_prediction.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img, highest_prediction.tag_name

def click_sign_send(sign_word):
    split_sign = split_hangul(sign_word)
    logger.info(sign_word + " " + str(split_sign))

    gallery_images = []
    sign_confirmed = []

    for sign in split_sign:
        sign_file_name = reverse_hangul_dict[sign]
        gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
        sign_confirmed.append(False)
    
    return gr.update(visible=False), gr.update(visible=True), gallery_images, sign_confirmed

# Custom Vision
def process_frame(frame, gallery_origin_images, sign_word, sign_confirmed):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame, tag_name = draw_boxes(frame, results.predictions)

        gallery_images = []

        split_sign = split_hangul(sign_word)
        sign_confirmed = sign_confirmed.replace("[","").replace("]", "")

        sign_confirmed_list = sign_confirmed.split(',')
        sign_confirmed_str_list = []

        for sign_confirmed in sign_confirmed_list:
            sign_confirmed = sign_confirmed.replace("'","").replace('"', "").strip()
            sign_confirmed_str_list.append(sign_confirmed)
            # logger.info(sign_confirmed)

        logger.info(sign_confirmed_str_list)

        confirmed_result= []

        for i, sign in enumerate(split_sign):
            sign_file_name = reverse_hangul_dict[sign]

            if sign_confirmed_str_list[i] == 'True':
                gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                confirmed_result.append("True")
            else:
                if sign_file_name == tag_name:
                    gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                    confirmed_result.append("True")
                else :    
                    gallery_images.append(f"./images/Basic/{sign_file_name}.jpg") 
                    confirmed_result.append("False")

        return annotated_frame, gallery_images, str(confirmed_result)
    
    except Exception as e:
        logger.error(f"Error during prediction: {e}")
        return frame, gallery_origin_images, sign_confirmed


# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            stop_button = gr.Button("Stop Speech")
        with gr.Column(visible=False) as column1:   # 기본적으로 숨김
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)
        with gr.Column(visible=False) as column2:   # 기본적으로 숨김
            with gr.Row():
                sign_input_textbox = gr.Textbox(label="지문자 입력 (2글자 단어)", value="", scale=7)
                sign_confirmed_textbox = gr.Textbox(show_label=False, visible=True)
                toggle_button = gr.Button("지문자 확인", scale=1)
            with gr.Row():  # Row layout for input and output
                gallery = gr.Gallery(columns=[6], rows=[1], show_label=False, show_share_button=False, show_download_button=False, interactive=False, show_fullscreen_button=False, height=130)
            with gr.Row():
                webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                webcam_output = gr.Image(label="Detected Objects")
            show_btn = gr.Button("힌트")
            webcam_input.stream(process_frame, inputs=[webcam_input, gallery, sign_input_textbox, sign_confirmed_textbox], outputs=[webcam_output, gallery, sign_confirmed_textbox])
            demo.title = "Azure Custom Vision Object Detection"
            demo.description = "Real-time object detection using Azure Custom Vision"
        
    with Modal(visible=False) as modal:  # 지문자 힌트 보여주기
            gr.Textbox("이미지가 보여질 공간")


    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, user_input, video_display, image_display, column1, column2])
    clear_button.click(
        lambda: ([], []),  # chatbot과 state를 모두 초기화
        inputs=None,
        outputs=[chatbot, state]  # chatbot과 state 모두 업데이트
    )
    stop_button.click(stop_speech)  # 음성을 멈추기
    show_btn.click(lambda: Modal(visible=True), None, modal)  # 지문자 힌트 보여주기
    toggle_button.click(fn=click_sign_send, inputs=[sign_input_textbox], outputs=[column1, column2, gallery, sign_confirmed_textbox])

# 실행
demo.launch(share=False)

In [2]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from gradio_modal import Modal
import cv2
from io import BytesIO
import requests
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from loguru import logger
import numpy as np
import io
import time

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Initialize the prediction client
ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
 
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}

reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}


###단어 자음/모음 분할 함수
def split_hangul(word):
    # 한글 초성, 중성, 종성 리스트
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
 
    separated = []
    for char in word:
        if '가' <= char <= '힣':  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성
            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])       # 중성 추가
            if final_consonants[final]:                   # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가
 
    return separated

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 음성 멈춤 기능을 위한 전역 변수
is_speaking = False

# 음성을 멈추는 함수
def stop_speech():
    global is_speaking
    if is_speaking:
        speech_synthesizer.stop_speaking_async()
        is_speaking = False
    return "Speech stopped."


def chat_with_openai(user_input, chat_history):
    global is_speaking
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()
        chat_history.append((user_input, assistant_reply))

        # 출처와 관련된 내용 확인
        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')

        # Azure Speech Service로 응답 읽기
        is_speaking = True
        speech_synthesizer.speak_text_async(assistant_reply)  # 여기에만 음성 합성 호출

        # return1을 반환 (예외가 발생하지 않은 경우)
        result1 = (chat_history, '', video_url, image_urls, gr.update(visible=True), gr.update(visible=False))
        return result1  # 정상 흐름에서는 return1

    except Exception as e:
        print(f"Error: {str(e)}")
        fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        chat_history.append((user_input, fallback_message))
        is_speaking = True
        
        # 예외 발생 시 fallback_message만 음성 합성
        speech_synthesizer.speak_text_async(fallback_message)  # 음성 합성 호출

        # return2를 반환 (예외 발생 시)
        result2 = (chat_history, '', None, [], gr.update(visible=False), gr.update(visible=True))
        return result2  # 예외 발생 시에는 return2


def draw_boxes(image, predictions):
    """가장 확률이 높은 객체만 경계 상자를 그리도록 수정"""
    img = image.copy()
    
    # 예측 결과 중 확률이 가장 높은 하나를 선택
    if predictions:
        highest_prediction = max(predictions, key=lambda p: p.probability)
        
        # if first_char_is_inprogress:
        # logger.info(str(highest_prediction.tag_name) + " " + str(highest_prediction.probability))
            # if highest_prediction.tag_name == first_char and highest_prediction.probability > 0.7:
                # global first_char_succeed
                # first_char_succeed = True
                # print("First character detected successfully!")
        
        # 확률이 0.5 이상인 객체만 선택
        if highest_prediction.probability > 0.5:
            color = (255, 0, 0)  # 경계 상자 색상 (빨간색)
            box = highest_prediction.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])

            # 경계 상자 그리기
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
            
            # 라벨과 확률 텍스트 추가
            label = f"{highest_prediction.tag_name}: {highest_prediction.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img, highest_prediction.tag_name

def click_sign_send(sign_word):
    split_sign = split_hangul(sign_word)
    logger.info(sign_word + " " + str(split_sign))

    gallery_images = []
    sign_confirmed = []

    for sign in split_sign:
        sign_file_name = reverse_hangul_dict[sign]
        gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
        sign_confirmed.append(False)
    
    return gr.update(visible=False), gr.update(visible=True), gallery_images, sign_confirmed

# Custom Vision
def process_frame(frame, gallery_origin_images, sign_word, sign_confirmed):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame, tag_name = draw_boxes(frame, results.predictions)

        gallery_images = []

        split_sign = split_hangul(sign_word)
        sign_confirmed = sign_confirmed.replace("[","").replace("]", "")

        sign_confirmed_list = sign_confirmed.split(',')
        sign_confirmed_str_list = []

        for sign_confirmed in sign_confirmed_list:
            sign_confirmed = sign_confirmed.replace("'","").replace('"', "").strip()
            sign_confirmed_str_list.append(sign_confirmed)
            # logger.info(sign_confirmed)

        logger.info(sign_confirmed_str_list)

        confirmed_result= []

        for i, sign in enumerate(split_sign):
            sign_file_name = reverse_hangul_dict[sign]

            if sign_confirmed_str_list[i] == 'True':
                gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                confirmed_result.append("True")
            else:
                if sign_file_name == tag_name:
                    gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                    confirmed_result.append("True")
                else :    
                    gallery_images.append(f"./images/Basic/{sign_file_name}.jpg") 
                    confirmed_result.append("False")

        return annotated_frame, gallery_images, str(confirmed_result)
    
    except Exception as e:
        logger.error(f"Error during prediction: {e}")
        return frame, gallery_origin_images, sign_confirmed

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            stop_button = gr.Button("Stop Speech")
        with gr.Column(visible=False) as column1:   # 기본적으로 숨김
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)
        with gr.Column(visible=False) as column2:   # 기본적으로 숨김
            with gr.Row():
                sign_input_textbox = gr.Textbox(label="지문자 입력 (2글자 단어)", value="", scale=7)
                sign_confirmed_textbox = gr.Textbox(show_label=False, visible=False)
                toggle_button = gr.Button("지문자 확인", scale=1)
                show_btn = gr.Button("힌트")
            with gr.Row():  # Row layout for input and output
                gallery = gr.Gallery(columns=[6], rows=[1], show_label=False, show_share_button=False, show_download_button=False, interactive=False, show_fullscreen_button=False, height=130)
            with gr.Row():
                webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                webcam_output = gr.Image(label="Detected Objects")
            
            webcam_input.stream(process_frame, inputs=[webcam_input, gallery, sign_input_textbox, sign_confirmed_textbox], outputs=[webcam_output, gallery, sign_confirmed_textbox])
            demo.title = "Azure Custom Vision Object Detection"
            demo.description = "Real-time object detection using Azure Custom Vision"
        
    with Modal(visible=False) as modal:  # 지문자 힌트 보여주기
            gr.Textbox("이미지가 보여질 공간")


    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, user_input, video_display, image_display, column1, column2])
    clear_button.click(
        lambda: ([], []),  # chatbot과 state를 모두 초기화
        inputs=None,
        outputs=[chatbot, state]  # chatbot과 state 모두 업데이트
    )
    stop_button.click(stop_speech)  # 음성을 멈추기
    show_btn.click(lambda: Modal(visible=True), None, modal)  # 지문자 힌트 보여주기
    toggle_button.click(fn=click_sign_send, inputs=[sign_input_textbox], outputs=[column1, column2, gallery, sign_confirmed_textbox])

# 실행
demo.launch(share=False)

c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\components\chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py:1776: UserWarning: A function (stop_speech) returned too many output values (needed: 0, returned: 1). Ignoring extra values.
    Output components:
        []
    Output values returned:
        ["Speech stopped."]
  warnings.warn(


Error: list index out of range


2024-12-23 09:41:02.951 | INFO     | __main__:click_sign_send:214 - 소맥 ['ㅅ', 'ㅗ', 'ㅁ', 'ㅐ', 'ㄱ']
2024-12-23 09:41:12.862 | INFO     | __main__:process_frame:259 - ['False', 'False', 'False', 'False', 'False']
2024-12-23 09:41:14.792 | INFO     | __main__:process_frame:259 - ['False', 'False', 'False', 'False', 'False']
2024-12-23 09:41:16.432 | INFO     | __main__:process_frame:259 - ['False', 'False', 'False', 'False', 'False']
2024-12-23 09:41:18.075 | INFO     | __main__:process_frame:259 - ['True', 'False', 'False', 'False', 'False']
2024-12-23 09:41:19.419 | INFO     | __main__:process_frame:259 - ['False', 'False', 'False', 'False', 'False']
2024-12-23 09:41:20.852 | INFO     | __main__:process_frame:259 - ['True', 'False', 'False', 'False', 'False']
2024-12-23 09:41:22.445 | INFO     | __main__:process_frame:259 - ['False', 'True', 'False', 'False', 'False']
2024-12-23 09:41:23.883 | INFO     | __main__:process_frame:259 - ['True', 'True', 'False', 'False', 'False']
2024-12-23 0

In [5]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from gradio_modal import Modal
import cv2
from io import BytesIO
import requests
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from loguru import logger
import numpy as np
import io
import time

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o-2")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-full-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Initialize the prediction client
ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
 
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

# 역딕셔너리 생성 (값 -> 키 변환용)
hangul_dict = {"Giyeok": "ㄱ", "Nieun": "ㄴ", "Digeut": "ㄷ", "Rieul": "ㄹ", "Mieum": "ㅁ", "Bieup": "ㅂ", "Siot": "ㅅ",
        "Ieung": "ㅇ", "Jieut": "ㅈ", "Chieut": "ㅊ", "Kieuk": "ㅋ", "Tieut": "ㅌ", "Pieup": "ㅍ", "Hieut": "ㅎ",
        "A": "ㅏ", "Ya": "ㅑ", "Eo": "ㅓ", "Yeo": "ㅕ", "O": "ㅗ", "Yo": "ㅛ", "U": "ㅜ", "Yu": "ㅠ", "Eu": "ㅡ",
        "Yi": "ㅣ", "Ae": "ㅐ", "Yae": "ㅒ", "E": "ㅔ", "Ye": "ㅖ", "Oe": "ㅚ", "Wi": "ㅟ", "Ui": "ㅢ"}

reverse_hangul_dict = {value: key for key, value in hangul_dict.items()}


###단어 자음/모음 분할 함수
def split_hangul(word):
    # 한글 초성, 중성, 종성 리스트
    initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
 
    separated = []
    for char in word:
        if '가' <= char <= '힣':  # 한글 음절인지 확인
            code = ord(char) - ord('가')
            initial = code // (21 * 28)  # 초성
            medial = (code % (21 * 28)) // 28  # 중성
            final = code % 28  # 종성
            separated.append(initial_consonants[initial])  # 초성 추가
            separated.append(medial_vowels[medial])       # 중성 추가
            if final_consonants[final]:                   # 종성이 있다면 추가
                separated.append(final_consonants[final])
        else:
            separated.append(char)  # 한글이 아닌 문자는 그대로 추가
 
    return separated

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 음성 멈춤 기능을 위한 전역 변수
is_speaking = False

# 음성을 멈추는 함수
def stop_speech():
    global is_speaking
    if is_speaking:
        speech_synthesizer.stop_speaking_async()
        is_speaking = False
    return "Speech stopped."


def chat_with_openai(user_input, chat_history):
    global is_speaking
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-full-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()
        

        print("확인", completion.choices[0].message.context['citations'])
        
        citations_context = completion.choices[0].message.context['citations']
        
        if len(citations_context) >0:
            citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
            video_url = citations[-2]
            image_urls = citations[-3].split(' ')
            chat_history.append((user_input, assistant_reply))

            # Azure Speech Service로 응답 읽기
            is_speaking = True
            speech_synthesizer.speak_text_async(assistant_reply)  # 여기에만 음성 합성 호출

            # return1을 반환 (예외가 발생하지 않은 경우)
            result1 = (chat_history, '', video_url, image_urls, gr.update(visible=True), gr.update(visible=False))
            return result1  # 정상 흐름에서는 return1
        else:
            fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
            chat_history.append((user_input, fallback_message))
            is_speaking = True
            
            
            # 예외 발생 시 fallback_message만 음성 합성
            speech_synthesizer.speak_text_async(fallback_message)  # 음성 합성 호출

            # return2를 반환 (예외 발생 시)
            result2 = (chat_history, '', None, [], gr.update(visible=False), gr.update(visible=True))
            return result2  # 예외 발생 시에는 return2

        # # # 출처와 관련된 내용 확인
        # # citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        # # video_url = citations[-2]
        # # image_urls = citations[-3].split(' ')
      
        # # Azure Speech Service로 응답 읽기
        # is_speaking = True
        # speech_synthesizer.speak_text_async(assistant_reply)  # 여기에만 음성 합성 호출

        # # return1을 반환 (예외가 발생하지 않은 경우)
        # result1 = (chat_history, '', video_url, image_urls, gr.update(visible=True), gr.update(visible=False))
        # return result1  # 정상 흐름에서는 return1

    except Exception as e:
        print(f"Error: {str(e)}")

        # fallback_message = "수어로 확인하기 어려운 단어입니다. 지문자로 연습해보세요!"
        # chat_history.append((user_input, fallback_message))
        # is_speaking = True
        
        # # 예외 발생 시 fallback_message만 음성 합성
        # speech_synthesizer.speak_text_async(fallback_message)  # 음성 합성 호출

        # # return2를 반환 (예외 발생 시)
        # result2 = (chat_history, '', None, [], gr.update(visible=False), gr.update(visible=True))
        # return result2  # 예외 발생 시에는 return2


def draw_boxes(image, predictions):
    """가장 확률이 높은 객체만 경계 상자를 그리도록 수정"""
    img = image.copy()
    
    # 예측 결과 중 확률이 가장 높은 하나를 선택
    if predictions:
        highest_prediction = max(predictions, key=lambda p: p.probability)
        
        # if first_char_is_inprogress:
        # logger.info(str(highest_prediction.tag_name) + " " + str(highest_prediction.probability))
            # if highest_prediction.tag_name == first_char and highest_prediction.probability > 0.7:
                # global first_char_succeed
                # first_char_succeed = True
                # print("First character detected successfully!")
        
        # 확률이 0.5 이상인 객체만 선택
        if highest_prediction.probability > 0.5:
            color = (255, 0, 0)  # 경계 상자 색상 (빨간색)
            box = highest_prediction.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])

            # 경계 상자 그리기
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
            
            # 라벨과 확률 텍스트 추가
            label = f"{highest_prediction.tag_name}: {highest_prediction.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img, highest_prediction.tag_name

def click_sign_send(sign_word):
    split_sign = split_hangul(sign_word)
    logger.info(sign_word + " " + str(split_sign))

    gallery_images = []
    sign_confirmed = []

    for sign in split_sign:
        sign_file_name = reverse_hangul_dict[sign]
        gallery_images.append(f"./images/Basic/{sign_file_name}.jpg")
        sign_confirmed.append(False)
    
    return gr.update(visible=False), gr.update(visible=True), gallery_images, sign_confirmed

# Custom Vision
def process_frame(frame, gallery_origin_images, sign_word, sign_confirmed):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame, tag_name = draw_boxes(frame, results.predictions)

        gallery_images = []

        split_sign = split_hangul(sign_word)
        sign_confirmed = sign_confirmed.replace("[","").replace("]", "")

        sign_confirmed_list = sign_confirmed.split(',')
        sign_confirmed_str_list = []

        for sign_confirmed in sign_confirmed_list:
            sign_confirmed = sign_confirmed.replace("'","").replace('"', "").strip()
            sign_confirmed_str_list.append(sign_confirmed)
            # logger.info(sign_confirmed)

        logger.info(sign_confirmed_str_list)

        confirmed_result= []

        for i, sign in enumerate(split_sign):
            sign_file_name = reverse_hangul_dict[sign]

            if sign_confirmed_str_list[i] == 'True':
                gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                confirmed_result.append("True")
            else:
                if sign_file_name == tag_name:
                    gallery_images.append(f"./images/Correct/{sign_file_name}.jpg") 
                    confirmed_result.append("True")
                else :    
                    gallery_images.append(f"./images/Basic/{sign_file_name}.jpg") 
                    confirmed_result.append("False")

        return annotated_frame, gallery_images, str(confirmed_result)
    
    except Exception as e:
        logger.error(f"Error during prediction: {e}")
        return frame, gallery_origin_images, sign_confirmed

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            stop_button = gr.Button("Stop Speech")
        with gr.Column(visible=False) as column1:   # 기본적으로 숨김
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)
        with gr.Column(visible=False) as column2:   # 기본적으로 숨김
            with gr.Row():
                sign_input_textbox = gr.Textbox(label="지문자 입력 (2글자 단어)", value="", scale=7)
                sign_confirmed_textbox = gr.Textbox(show_label=False, visible=False)
                toggle_button = gr.Button("지문자 확인", scale=1)
                show_btn = gr.Button("힌트")
            with gr.Row():  # Row layout for input and output
                gallery = gr.Gallery(columns=[6], rows=[1], show_label=False, show_share_button=False, show_download_button=False, interactive=False, show_fullscreen_button=False, height=130)
            with gr.Row():
                webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
                webcam_output = gr.Image(label="Detected Objects")
            
            webcam_input.stream(process_frame, inputs=[webcam_input, gallery, sign_input_textbox, sign_confirmed_textbox], outputs=[webcam_output, gallery, sign_confirmed_textbox])
            demo.title = "Azure Custom Vision Object Detection"
            demo.description = "Real-time object detection using Azure Custom Vision"
        
    with Modal(visible=False) as modal:  # 지문자 힌트 보여주기
        gr.Image(value="images/hint.jpg", label="지문자 힌트")  # 이미지 경로에 맞게 설정


    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, user_input, video_display, image_display, column1, column2])
    clear_button.click(
        lambda: ([], []),  # chatbot과 state를 모두 초기화
        inputs=None,
        outputs=[chatbot, state]  # chatbot과 state 모두 업데이트
    )
    stop_button.click(stop_speech)  # 음성을 멈추기
    show_btn.click(lambda: Modal(visible=True), None, modal)  # 지문자 힌트 보여주기
    toggle_button.click(fn=click_sign_send, inputs=[sign_input_textbox], outputs=[column1, column2, gallery, sign_confirmed_textbox])

# 실행
demo.launch(share=False)

c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\components\chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


확인 []
확인 [{'content': '기타\n오른 주먹의 1지를 펴서 반쯤 구부려 끝이 오른쪽 귀로 향하게 하여 좌우로 두 번 움직인 다음, 펴서 세운 왼 주먹의 5지 등에 오른 주먹의 1·2지를 펴서 등이 위로 향하게 하여 대고 왼쪽에서 오른쪽으로 움직인다.\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151230/237175/IMG000232857_700X466.jpg http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151230/237176/IMG000232858_700X466.jpg\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20220801/1004437/MOV000359630_700X466.mp4\n정보', 'title': None, 'url': None, 'filepath': None, 'chunk_id': '0'}, {'content': '기타\n모로 세운 왼 주먹의 밑면을, 오른 주먹의 1지를 펴서 윗면으로 스쳐 낸다.\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151217/229384/IMG000222762_700X466.jpg\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191029/632415/MOV000250158_700X466.mp4\n의미', 'title': None, 'url': None, 'filepath': None, 'chunk_id': '0'}]
